In [20]:
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
import os

In [17]:
import subprocess
import socket

# Get Windows host IP from WSL
def get_windows_host_ip():
    # Method 1: Try WSL2 bridge interface (default route)
    try:
        result = subprocess.run(
            ["ip", "route", "show", "default"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'default' in line and 'via' in line:
                parts = line.split()
                via_index = parts.index('via')
                ip = parts[via_index + 1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (WSL default gateway)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 1 failed: {e}")
    
    # Method 2: Try /etc/resolv.conf nameserver
    try:
        result = subprocess.run(
            ["cat", "/etc/resolv.conf"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'nameserver' in line:
                ip = line.split()[1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (resolv.conf)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 2 failed: {e}")
    
    # Fallback to localhost
    print("⚠ Falling back to localhost")
    return "127.0.0.1"
    
WINDOWS_HOST_IP = get_windows_host_ip()

✓ Found Ollama at 172.17.96.1 (WSL default gateway)


In [ ]:
EMBEDDING = OllamaEmbeddings(base_url=f"http://{WINDOWS_HOST_IP}:11434",
                             model="embeddinggemma:300m")
vector_store = Chroma(
    collection_name="eda-hospital",
    embedding_function=EMBEDDING,
    persist_directory="vectordb",
    create_collection_if_not_exists=False,
)

In [28]:
for folder in os.listdir("./assets/output"):
    for file in os.listdir(f"./assets/output/{folder}"):
        if file.endswith("-ENG.md"):
            print(file)
            with open(f"./assets/output/{folder}/{file}") as f:
                context = f.read()
                print(context)

HA-4-0009(1) Understanding Epidural Patient-Controlled Analgesia_0-ENG.md
Understanding

# Epidural Patient Controlled Analgesia

Epidural Patient Controlled Analgesia

Stay away from pain

## Pain Relief Benefits

~ Anesthesia Team ~

Let you feel assured with pain relief

## What is Epidural Patient Controlled Analgesia

An anesthesiologist temporarily places a catheter in the patient's epidural space on the back, then connects a machine to the catheter. The machine continuously administers pain medication, allowing the anesthetic pain medication to enter the epidural space through the catheter, locally blocking the nerves that transmit pain signals, thereby achieving pain relief. When the patient feels pain, they can press the control button themselves to supplement the medication.

<div style="text-align: center;"><img src="imgs/img_in_image_box_678_421_997_601.jpg" alt="Image" width="18%" /></div>

This is an advanced medication machine. When you feel pain, you can press the medic

In [ ]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"baz": "bar"})
document_2 = Document(page_content="thud", metadata={"bar": "baz"})
document_3 = Document(page_content="i will be deleted :(")

documents = [document_1, document_2, document_3]
ids = ["1", "2", "3"]
vector_store.add_documents(documents=documents, ids=ids)
